In [1]:
# =========================
# INSTALL DEPENDENCIES
# =========================
!pip install -q google-generativeai requests


In [ ]:
# =========================
# CONFIGURATION (EDIT THIS)
# =========================

# 🔑 Gemini API Key (from Google AI Studio)
GEMINI_API_KEY = "MY_GEMINI_API_KEY_HERE"

# 🔑 GitHub Personal Access Token
GITHUB_TOKEN = "MY_GITHUB_TOKEN_HERE"

# 📦 GitHub Repository Info
GITHUB_USERNAME = "aya-zid"
REPOSITORY_NAME = "simple-math-utils"
PYTHON_FILE_PATH = "math_utils.py"


In [10]:
# =========================
# FETCH FILE FROM GITHUB
# =========================
import requests
github_url = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPOSITORY_NAME}/contents/{PYTHON_FILE_PATH}"

headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3.raw"
}

response = requests.get(github_url, headers=headers)

if response.status_code != 200:
    raise Exception("❌ Failed to fetch file from GitHub")

python_code = response.text
print("✅ Successfully fetched math_utils.py from GitHub")



✅ Successfully fetched math_utils.py from GitHub


In [6]:
# =========================
# GEMINI CONFIGURATION
# =========================
import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-2.5-flash")


In [11]:
# =========================
# PROMPT FOR AI AGENT
# =========================
prompt = f"""
You are a senior Python developer and software tester.

You are given a Python file from a GitHub repository.

TASKS:
1. Generate a pytest test file named `test_math_utils.py`
2. Generate a professional `README.md`

TEST REQUIREMENTS:
- Use pytest
- Cover normal cases
- Cover edge cases
- Test exceptions

README REQUIREMENTS:
- Project description
- Installation instructions
- How to run tests

OUTPUT FORMAT (STRICT):
--- test_math_utils.py ---
<pytest file content>

--- README.md ---
<README content>

HERE IS THE PYTHON FILE:
{python_code}
"""

In [12]:

# =========================
# CALL GEMINI
# =========================
response = model.generate_content(prompt)

print("\n==============================")
print("🤖 AI GENERATED FILES")
print("==============================\n")
print(response.text)



🤖 AI GENERATED FILES

--- test_math_utils.py ---
import pytest
from math_utils import add, subtract, multiply, divide

# Assume the provided functions are in a file named `math_utils.py` in the same directory.

# --- Tests for add function ---
def test_add_normal_cases():
    """Test add function with various normal inputs."""
    assert add(1, 2) == 3
    assert add(-1, -2) == -3
    assert add(0, 0) == 0
    assert add(5.5, 4.5) == 10.0
    assert add(-10, 5) == -5
    assert add(1000, 2000) == 3000

def test_add_edge_cases():
    """Test add function with edge cases like large numbers and float precision."""
    # Large numbers
    assert add(1_000_000_000, 2_000_000_000) == 3_000_000_000
    # Floating point precision (use pytest.approx for comparison)
    assert add(0.1, 0.2) == pytest.approx(0.3)
    assert add(123.4567, 789.0123) == pytest.approx(912.469)
    # Mixed large and very small numbers
    assert add(1_000_000_000.0, 0.000000001) == pytest.approx(1_000_000_000.0000000

In [14]:
ai_output = response.text

# Split AI output
test_part = ai_output.split("--- test_math_utils.py ---")[1].split("--- README.md ---")[0].strip()
readme_part = ai_output.split("--- README.md ---")[1].strip()

print("✅ Files extracted from AI output")


✅ Files extracted from AI output


In [15]:
import base64


In [16]:
def push_file_to_github(repo, owner, path, content, message):
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"

    encoded_content = base64.b64encode(content.encode()).decode()

    data = {
        "message": message,
        "content": encoded_content
    }

    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }

    response = requests.put(url, json=data, headers=headers)

    if response.status_code in [200, 201]:
        print(f"✅ {path} pushed successfully")
    else:
        print(f"❌ Failed to push {path}")
        print(response.json())


In [17]:
push_file_to_github(
    repo=REPOSITORY_NAME,
    owner=GITHUB_USERNAME,
    path="test_math_utils.py",
    content=test_part,
    message="Add pytest tests generated by AI"
)


✅ test_math_utils.py pushed successfully


In [18]:
push_file_to_github(
    repo=REPOSITORY_NAME,
    owner=GITHUB_USERNAME,
    path="README.md",
    content=readme_part,
    message="Add README generated by AI"
)


✅ README.md pushed successfully
